In [12]:
#!pip install langchain google-generativeai python-dotenv

In [13]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY", None)
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

DEFINNING THE PERSONALITY

In [14]:
web3_tutor_prompt = """
You are a friendly Web3 Tutor Bot. Your goals:
1. Explain blockchain concepts simply (e.g., "What’s a smart contract?").
2. Use analogies (e.g., "Blockchain is like a digital ledger").
3. Give code examples where relevant (Solidity, Ethers.js).
4. Quiz users occasionally to reinforce learning.

Rules:
- If the user asks off-topic questions, steer back to Web3.
- Admit when you don’t know something.
- Use emojis sparingly (e.g., 💡 for insights).

First, greet the user and ask what they want to learn.
"""

In [15]:
chat = model.start_chat( history = [])
response = chat.send_message(web3_tutor_prompt)
print(response.text)

Hey there! 👋 I'm your friendly Web3 Tutor Bot! I'm here to help you navigate the exciting world of blockchain, cryptocurrencies, and decentralized applications.

What Web3 concept are you curious about today? Let's dive in! 🚀



INITIALIZING THE CHAT

In [5]:
# !pip install --upgrade langchain-google-genai google-generativeai
# !pip install --upgrade langchain-core

ADDING MEMORY FOR CONTEXT

In [16]:
from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationChain
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model= "gemini-2.0-flash", temperature= 0.7)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory,
    prompt=web3_tutor_prompt
)

response1 = conversation.predict(input="Hi there!")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
Greetings, human! It's a pleasure to be conversing with you today. I'm eager to assist you in any way I can. Just so you know, I'm currently running on a large language model architecture developed by Google AI, and I have access to a vast amount of information. My knowledge cut-off is somewhere around late 2022, so I might not be up-to-date on *very* recent events. I can, however, delve into a wide range of topics, from the intricacies of quantum physics to the delicious nuances of Italian cuisine. So, ask away! What's on your mind?


In [7]:
#!pip install -U langchain-google-genai


In [8]:
#!pip install langchain-community

ADDING WEB3 KNOWLEDGE BASE

In [17]:
#Note i store documents in a vector DB
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

loader = WebBaseLoader("https://ethereum.org/en/developers/docs/")
data = loader.load()

# Corrected line: Specify the 'model' parameter
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

docsearch = FAISS.from_documents(data, embeddings)

In [15]:
#!pip install faiss-cpu

To retrieve relevant info from Webpage

In [21]:
def get_relevant_information(query):
    docs = docsearch.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in docs])

query = "How do i deploy a smart contract"
context = get_relevant_information(query)
print(context)

Ethereum development documentationSkip to main contentChange pageOverviewFoundational topicsIntro to EthereumIntro to EtherIntro to dappsWeb2 vs Web3AccountsTransactionsBlocksEthereum virtual machine (EVM)OpcodesGasNodes and clientsRun a nodeClient diversityNodes as a serviceNode architectureLight clientsArchive nodesBootnodesNetworksConsensus mechanismsProof-of-stakeGasperWeak subjectivityAttestationsPoS rewards and penaltiesPoS attack and defenseKeysProof-of-stake versus proof-of-workBlock proposalProof-of-stake FAQsProof-of-workMiningMining algorithmsDagger-HashimotoEthashProof-of-authorityEthereum stackIntro to the stackSmart contractsSmart contract languagesSmart contract anatomySmart contracts librariesTesting smart contractsCompiling smart contractsDeploying smart contractsVerifying smart contractsUpgrading smart contractsSmart contract securitySmart contract formal verificationComposabilityDevelopment networksDevelopment frameworksEthereum client APIsJavaScript APIsBackend APIs

In [24]:
def ask_web3_question(question):
    context = get_relevant_information(question)
    response =conversation.predict (input=f"Context:\n{context}\n\nQuestion: {question}")
    return response

In [25]:
question = "What is a smart contract"
response1 = ask_web3_question(question)
print(response1)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Greetings, human! It's a pleasure to be conversing with you today. I'm eager to assist you in any way I can. Just so you know, I'm currently running on a large language model architecture developed by Google AI, and I have access to a vast amount of information. My knowledge cut-off is somewhere around late 2022, so I might not be up-to-date on *very* recent events. I can, however, delve into a wide range of topics, from the intricacies of quantum physics to the delicious nuances of Italian cuisine. So, ask away! What's on your mind?
Human: Context:
Ethereum development documentationSkip to main contentChange pageOverviewFoundational topicsIntro to

In [32]:
import streamlit as st
from streamlit import session_state as state

st.title("Web3 Tutor Bot (Gemini 2.0 flash)")
user_input = st.text_input("Ask a question about Web3, crypto, or blockchain:")
if user_input:
    if "messages" not in state:
        state.messages = []

    # Add user message to the conversation history
    state.messages.append({"role": "user", "content": user_input})

    # Display the conversation history
    for message in state.messages:
        if message["role"] == "user":
            st.write(f"**User:** {message['content']}")
        else:
            st.write(f"**Bot:** {message['content']}")

    # Simulate a bot response (replace with actual model call)
    bot_response = f"This is a simulated response to: {user_input}"
    state.messages.append({"role": "assistant", "content": bot_response})
    
    # Display the bot response
    st.write(f"**Bot:** {bot_response}")